# 1. Preprocessing

## 1.1 Loading libraries and dataset

In [1]:
import json
import os
from pathlib import Path
from os.path import join
from os import listdir

import kagglehub
import pandas as pd
import numpy as np
from tqdm import tqdm

os.chdir('/teamspace/studios/this_studio/ai-capstone-proj')
print(os.getcwd())

import src.utils as u
import src.constants as c
import src.balanced.dataset_processing as bdp



/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/teamspace/studios/this_studio/ai-capstone-proj


/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: '2.0.1' (you have '2.0.0'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
api.model_list_cli()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/kaggle/api/kaggle_api.py:1055: SyntaxWarning: invalid escape sequence '\&'
  """Creates (aka \&quot;drops\&quot;) a new file into the inbox.  # noqa: E501
/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/kaggle/api/kaggle_api.py:1076: SyntaxWarning: invalid escape sequence '\&'
  """Creates (aka \&quot;drops\&quot;) a new file into the inbox.  # noqa: E501
/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/kaggle/api_client.py:244: SyntaxWarning: invalid escape sequence '\['
  sub_kls = re.match('list\[(.*)\]', klass).group(1)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/kaggle/api_client.py:249: SyntaxWarning: invalid escape sequence '\('
  sub_kls = re.match('dict\(([^,]*), (.*)\)', klass).group(2)


ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json', 'Date': 'Sun, 26 Jan 2025 00:35:49 GMT', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Origin': '*', 'Set-Cookie': 'ka_sessionid=03fd0c9afdf9a23fb3c6df596e5cab0e; max-age=2626560; path=/, GCLB=COb55tGfqrmZjAEQAw; path=/; HttpOnly', 'Vary': 'Accept-Encoding', 'X-Kaggle-MillisecondsElapsed': '1', 'X-Kaggle-RequestId': '18d745b9bdbc5940b032da9f0dfe3ca9', 'X-Kaggle-ApiVersion': '1.6.17', 'X-Kaggle-HubVersion': '0.3.6', 'X-Frame-Options': 'SAMEORIGIN', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains; preload', 'Content-Security-Policy': "object-src 'none'; script-src 'nonce-fWSCbALfy7+Oz6hBzqNO1w==' 'report-sample' 'unsafe-inline' 'unsafe-eval' 'strict-dynamic' https: http:; base-uri 'none'; report-uri https://csp.withgoogle.com/csp/kaggle/20201130; frame-src 'self' https://www.kaggleusercontent.com https://www.youtube.com/embed/ https://p

In [5]:
cwd = Path().absolute().parent

kagglehub.login()
kagglepath = kagglehub.dataset_download('arct22/codebrim-balanced')

org_imgs_merged = join(cwd, 'datasets', 'org', 'images')
pad_images = join(cwd, 'datasets', 'pad', 'images')
pad_labels = join(cwd, 'datasets', 'pad', 'labels')
print(kagglepath)

ImportError: You need the `ipywidgets` module: `pip install ipywidgets`.

In [ ]:
def list_all_folders(path):
    base_path = Path(path)
    return [str(p.relative_to(base_path)) for p in base_path.rglob("*") if p.is_dir()]


list_all_folders(kagglepath)

We can see that dataset subdirectories train, test and val have separate subfolders for backgrounds and defects. We will merge them, since YOLO will be not able to recognize such folder structure.

In [ ]:
copiedpaths = bdp.copy_bg_and_defects(kagglepath, org_imgs_merged)

In [ ]:
bg_xml = u.xml_to_dict(join(kagglepath, 'metadata', 'background.xml'))['Annotation']['Defect']
defects_xml = u.xml_to_dict(join(kagglepath, 'metadata', 'defects.xml'))['Annotation']['Defect']

org_bg_df = pd.DataFrame(bg_xml)
org_defects_df = pd.DataFrame(defects_xml)
org_df = pd.concat([org_bg_df, org_defects_df])

org_df = org_df.rename(columns={'@name': c.IMG})
org_df[c.defect_names] = org_df[c.defect_names].astype(np.uint8)
org_df

## 1.2 Data Exploration

In [ ]:
bdp.plot_corr_matrix(org_df, (8, 6))

The correlation matrix reveals logical relationships between defect classes- for ex. there is no corrosion stains without exposed reinforcement bars, and bars can be only exposed, if conrete is spalted. This indicates, that data collection methodology was collected correctly, showing that images were correctly annotated.

We can see that maximal correlation for between features exists for pair Spallation - ExposedBars- 0.787. In addition, we can also see high correlation between those two defect types and CorrosionStain (0.478 and 0.523, respectivelly). Again,

We will merge these three columns to reduce number of classes that model will have to detect.

In [ ]:
merged_cols = [c.SPALLATION, c.EXPOSEDBARS, c.CORROSIONSTAIN]

merged_col_name = "_".join(col for col in merged_cols)
org_df[merged_col_name] = (org_df[merged_cols[0]] | org_df[merged_cols[1]] | org_df[merged_cols[2]])
bdp.plot_corr_matrix(org_df, (8, 6))

Our new feature strongly correlates with original features, what indicates that it is able to correctly capture relationships between these defect types. Thus, we can safely drop original columns.

In [ ]:
org_df = org_df.drop(merged_cols, axis=1)
bdp.plot_corr_matrix(org_df, (8, 6))

By applying dimensionality reduction, we were able to reduce amount of possible combinations from 33 to 9.

Let's see now how many times each combination of defects appears in dataset.

In [ ]:
numerical_cols = org_df.select_dtypes(include=np.number).columns.tolist()

combination_counts = org_df.groupby(numerical_cols).size().reset_index(name='count')

# Sort by count in descending order
combination_counts = combination_counts.sort_values('count', ascending=False)
combination_counts

We can see that three is combination [0, 0, 0, 0] which is rarest of all combinations (only 52 times). Since this is undefined type of defect, we will drop rows with these values.

In [ ]:
rows_to_drop = ~(org_df[numerical_cols] == [0, 0, 0, 0]).all(axis=1)
org_df_filtered = org_df[rows_to_drop].copy()

Now, we will map a numerical value to each combination.

In [ ]:
# First, make sure to get the combinations correctly
combinations = org_df_filtered[numerical_cols].value_counts().index.tolist()

# Print to check what we have
print("Combinations:", combinations)

# Only then create the mapping
mapping = {comb: i for i, comb in enumerate(combinations)}

# Create the new column
org_df_filtered['combination_id'] = org_df_filtered[numerical_cols].apply(tuple, axis=1).map(mapping)

In [ ]:
org_df_filtered[numerical_cols].sum()

And map each label to adequate image in train, test, or split folder.

In [ ]:
splitted_dfs = {
    'train': [],
    'test': [],
    'val': []
}

for split in ['val', 'test', 'train']:
    splitted_imgs = listdir(join(org_imgs_merged, split))
    splitted_dfs[split] = org_df_filtered[org_df_filtered['img'].isin(splitted_imgs)]


In [ ]:
yolo_labels_new = {
    'train': {
        'img': [],
        'bbox': []
    },
    'test': {
        'img': [],
        'bbox': []
    },
    'val': {
        'img': [],
        'bbox': []
    }
}

In [ ]:
for split in ['val', 'test', 'train']:
    org_dir = join(org_imgs_merged, split)

    output_path = join(pad_images, split)

    if os.path.exists(output_path):
        print(f'Skipping resizing and padding, path {output_path} already exists')
        continue

    os.makedirs(output_path)

    print(f'resizing and padding: {split} dataset')
    for img in tqdm(dfs_splited[split]['img']):
        bbox = bdp.resize_and_pad_yolo(image_path=join(org_dir, img),
                                       output_path=join(output_path, img),
                                       target_size=640)

        yolo_labels_new[split][c.IMG].append(img)
        yolo_labels_new[split][c.BBOX].append(bbox)

In [ ]:
yolo_labels_new_filepath = '/teamspace/studios/this_studio/datasets/pad/labels/yolo_labels_new.json'

In [ ]:
if not os.path.exists(yolo_labels_new_filepath):
    with open(yolo_labels_new_filepath, 'w') as file:
        json.dump(yolo_labels_new, file, indent=2)

In [ ]:
with open(yolo_labels_new_filepath, 'r') as f:
    yolo_labels_new = json.load(f)

yolo_df = {}

In [ ]:
for split in ['test']:
    split_df = pd.DataFrame(yolo_labels_new[split])
    copydf = splitted_dfs[split].copy()
    copydf['bbox'] = copydf['img'].map(split_df.set_index('img')['bbox'])
    print(copydf)

In [ ]:
import os
from os.path import join

non_iterable_objects = []

for split in ['train', 'test', 'val']:
    directory = join(cwd, 'datasets', 'pad', 'labels', split)
    os.makedirs(directory, exist_ok=True)
    
    for index, row in yolo_df[split].iterrows():
        filename = row['img'].split('.')[0] + '.txt'
        label_class = row['combination_id']

        print(row['bbox'])
        


print(len(non_iterable_objects))

In [ ]:
for split in ['train', 'test', 'val']:
    directory = join(cwd, 'datasets', 'pad', 'labels', split)
    os.makedirs(directory, exist_ok=True)
    
    for index, row in yolo_df[split].iterrows():
        filename = row['img'].split('.')[0] + '.txt'
        print(f"Filename: {filename}")
        print(f"bbox value: {row['bbox']}")
        print(f"bbox type: {type(row['bbox'])}")
        break  # Just look at first row to understand the data structure

In [ ]:
from ultralytics import YOLO
import os

os.chdir('/teamspace/studios/this_studio')

model = YOLO('yolo11n.pt')

model.train(data="yolo_train.yaml", epochs=100, imgsz=640, device=0, batch=128)

In [ ]:
from lightning_sdk import Studio
s = Studio()
s.stop()